In [1]:
print('start ch uploads')
#Clickhouse db w/ Goldsky
# https://clickhouse.com/docs/en/integrations/python

import requests as r
import pandas as pd
import clickhouse_connect as cc
import os

import sys
sys.path.append("../helper_functions")
import duneapi_utils as d
import pandas_utils as p
import clickhouse_utils as ch
import csv_utils as cu
import google_bq_utils as bqu
import opstack_metadata_utils as ops
import goldsky_db_utils as gsb
sys.path.pop()

import time

start ch uploads


In [2]:
client = ch.connect_to_clickhouse_db() #Default is OPLabs DB
# client.close()

table_name = 'daily_aggegate_l2_chain_usage_goldsky'

In [3]:
# bqu.delete_bq_table('api_table_uploads',table_name)

In [4]:
trailing_days = 3 #7 #90
max_execution_secs = 5000

In [5]:
# chain_mappings_list = [
#     # {'schema_name': 'zora', 'display_name': 'Zora', 'has_blob_fields': True},
#     # {'schema_name': 'pgn', 'display_name': 'Public Goods Network', 'has_blob_fields': True},
#     # {'schema_name': 'base', 'display_name': 'Base', 'has_blob_fields': False},
#     # {'schema_name': 'op', 'display_name': 'OP Mainnet', 'has_blob_fields': True},
#     {'schema_name': 'mode', 'display_name': 'Mode', 'has_blob_fields': True},
#     {'schema_name': 'metal', 'display_name': 'Metal', 'has_blob_fields': True},
#     {'schema_name': 'fraxtal', 'display_name': 'Fraxtal', 'has_blob_fields': True},
#     # {'schema_name': 'bob', 'display_name': 'BOB (Build on Bitcoin)', 'has_blob_fields': False},
#     {'schema_name': 'cyber', 'display_name': 'Cyber', 'has_blob_fields': True},
#     {'schema_name': 'mint', 'display_name': 'Mint', 'has_blob_fields': True},
#     # Add more mappings as needed
# ]
# chain_mappings_dict = {item['schema_name']: item['display_name'] for item in chain_mappings_list}


In [6]:
# Get Chain List
chain_configs = ops.get_superchain_metadata_by_data_source('oplabs') # OPLabs db

# if we want to filter for backfills
# chain_configs = chain_configs[chain_configs['blockchain'].isin(['swan','lisk'])]
chain_configs = chain_configs[chain_configs['blockchain'] == 'bob']

chain_configs

,chain_name,display_name,mainnet_chain_id,chain_layer,alignment,da_layer,output_root_layer,gas_token,block_time_sec,blockchain
20,bob,BOB (Build on Bitcoin),60808,L2,OP Chain,ethereum bitcoin,ethereum,ETH,2.0,bob


In [7]:
sql_directory = "inputs/sql/"

query_names = [
        # Must match the file name in inputs/sql
        "ch_template_alltime_chain_activity"
]

In [8]:
# Set the memory limit using the settings parameter
settings = {
    'max_memory_usage': 200000000000  # 200GB in bytes
}


In [9]:
unified_dfs = []
for qn in query_names:
        for index, chain in chain_configs.iterrows():
                chain_schema = chain['blockchain']
                display_name = chain['display_name']
                block_time_sec = chain['block_time_sec']
                # Read the SQL query from file
                with open(os.path.join(sql_directory, f"{qn}.sql"), "r") as file:
                        query = file.read()
                print(qn + ' - ' + chain_schema)

                #Pass in Params to the query
                query = query.replace("@chain_db_name@", chain_schema)
                query = query.replace("@trailing_days@", str(trailing_days))
                query = query.replace("@block_time_sec@", str(block_time_sec))
                query = query.replace("@max_execution_secs@", str(max_execution_secs))


                query = gsb.process_goldsky_sql(query)

                #Save the query
                output_folder = os.path.join("outputs", "sql")
                os.makedirs(output_folder, exist_ok=True)
                filename = f"{chain_schema}_query.sql"
                file_path = os.path.join(output_folder, filename)
                with open(file_path, 'w') as file:
                        file.write(query)

                # Execute the query
                result_df = client.query_df(query, settings=settings)
        #         # Write to csv
        #         df.to_csv('outputs/chain_data/' + qn + '.csv', index=False)
        #         # print(df.sample(5))
        #         time.sleep(1)
                if not result_df.empty:
                        # print(query)
                        # display(result_df)
                        result_df['chain_raw'] = result_df['chain']
                        result_df['chain'] = display_name
                        result_df['chain_id'] = result_df['chain_id'].astype('string')

                        #BQ Upload
                        bqu.append_and_upsert_df_to_bq_table(result_df, table_name, unique_keys = ['dt','chain','network'])

                        unified_dfs.append(result_df)
                else:
                        print('dataframe is empty')

        write_df = pd.concat(unified_dfs)
        write_df.to_csv('outputs/chain_data/' + qn + '.csv', index=False)
        
        # # # Print the results

ch_template_alltime_chain_activity - bob
SELECT *
    , num_blocks * block_time_sec as active_secs_per_day
    , l2_num_txs_per_day / num_blocks AS l2_num_txs_per_day_per_block
    , l2_num_txs_per_day / (num_blocks*block_time_sec) AS num_user_txs_per_second
    , l2_gas_used / num_blocks AS l2_gas_used_per_block
    , l2_gas_used / (num_blocks*block_time_sec) AS l2_gas_used_per_second
    , l2_eth_fees_per_day / num_blocks AS l2_eth_fees_per_block
    , l2_eth_fees_per_day / (num_blocks*block_time_sec) AS l2_eth_fees_per_second
    , chain as chain_name

FROM (
    SELECT
        DATE_TRUNC('day', toDateTime(t.block_timestamp)) AS dt,
        chain, network, chain_id, cast(2.0 as Float64) AS block_time_sec,

        COUNT(*) AS num_raw_txs,
        1+ (MAX(t.block_number) - MIN(t.block_number)) AS num_blocks,

        SUM(IF(gas_price = 0 AND to_address = '0x4200000000000000000000000000000000000015', 1, 0)) AS l2_num_attr_deposit_txs_per_day,
        SUM(IF(gas_price = 0 AND to_addres

,dt,chain,network,chain_id,block_time_sec,num_raw_txs,num_blocks,l2_num_attr_deposit_txs_per_day,l2_num_user_deposit_txs_per_day,l2_num_txs_per_day,...,avg_l1_fee_scalar,avg_l1_blob_fee_scalar,active_secs_per_day,l2_num_txs_per_day_per_block,num_user_txs_per_second,l2_gas_used_per_block,l2_gas_used_per_second,l2_eth_fees_per_block,l2_eth_fees_per_second,chain_name
0,2024-05-14,bob,mainnet,60808,2.0,60197,43200,43200,1177,15819,...,684000.000000,0.000000,86400.0,0.366181,0.183090,9.860665e+04,49303.325567,0.000026,0.000013,bob
1,2024-06-20,bob,mainnet,60808,2.0,69980,43200,43200,57,26723,...,20000.000000,611590.000000,86400.0,0.618588,0.309294,1.670282e+05,83514.092269,0.000012,0.000006,bob
2,2024-06-10,bob,mainnet,60808,2.0,59461,43200,43200,130,16131,...,20000.000000,611590.000000,86400.0,0.373403,0.186701,1.052782e+05,52639.120995,0.000007,0.000003,bob
3,2024-06-18,bob,mainnet,60808,2.0,68866,43200,43200,75,25590,...,20000.000000,611590.000000,86400.0,0.592361,0.296181,2.103924e+05,105196.218067,0.000010,0.000005,bob
4,2024-04-22,bob,mainnet,60808,2.0,43756,43200,43200,67,489,...,684000.000000,0.000000,86400.0,0.011319,0.005660,5.074272e+04,25371.362025,0.000006,0.000003,bob
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,2024-05-28,bob,mainnet,60808,2.0,110344,43200,43200,172,66972,...,68400.000000,20000.000000,86400.0,1.550278,0.775139,5.816384e+05,290819.179190,0.000086,0.000043,bob
68,2024-05-19,bob,mainnet,60808,2.0,92963,43200,43200,467,49296,...,684000.000000,0.000000,86400.0,1.141111,0.570556,3.552055e+05,177602.739931,0.000071,0.000035,bob
69,2024-05-21,bob,mainnet,60808,2.0,102404,43200,43200,237,58967,...,252306.415453,14025.132701,86400.0,1.364977,0.682488,3.823373e+05,191168.669907,0.000086,0.000043,bob
70,2024-05-17,bob,mainnet,60808,2.0,121604,43200,43200,1028,77375,...,684000.000000,0.000000,86400.0,1.791088,0.895544,1.269893e+06,634946.527361,0.000294,0.000147,bob


Start Writing api_table_uploads.daily_aggegate_l2_chain_usage_goldsky_staging
Data loaded successfully to api_table_uploads.daily_aggegate_l2_chain_usage_goldsky_staging


2024-08-28 14:53:52,563 INFO google_bq_utils Append and upsert to api_table_uploads.daily_aggegate_l2_chain_usage_goldsky completed successfully
2024-08-28 14:53:52,779 INFO google_bq_utils Staging table oplabs-tools-data.api_table_uploads.daily_aggegate_l2_chain_usage_goldsky_staging deleted.


In [11]:
# Copy to Dune
print('upload bq to dune')
sql = '''
SELECT *
FROM `api_table_uploads.daily_aggegate_l2_chain_usage_goldsky`
'''
bq_df = bqu.run_query_to_df(sql)

dune_table_name = 'ch_template_alltime_chain_activity'
d.write_dune_api_from_pandas(bq_df, dune_table_name,table_description = dune_table_name)

upload bq to dune
uploading ch_template_alltime_chain_activity
Response status code: 200
Response content: b'{"success":true,"table_name":"ch_template_alltime_chain_activity"}'
table at: dune.oplabspbc.dataset_ch_template_alltime_chain_activity
